In [ ]:
from nltk import word_tokenize, ngrams
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
from google.colab import files
from nltk import word_tokenize, ngrams
import torch
import random


import torch.nn as nn

target = torch.tensor([1]).long()
print(target)

predicted = torch.tensor( [-2.5, -4.212] ).float()
print(predicted)
print(predicted.view(1,2))
lossfxn = nn.CrossEntropyLoss()
loss = lossfxn(predicted.view(1,2), target)
print(loss) # outputs tensor(2.4402)

#predicted = torch.tensor([[1,10,3,4]]).float()
#target = torch.tensor([1]).long()
#lossfxn = nn.CrossEntropyLoss()
#loss = lossfxn(predicted, target)
#print(loss) # outputs tensor(0.0035)


class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim, output_dim)

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        # Linear function  # LINEA
      out = self.fc1(x)
      out = self.sigmoid(out)
      out = self.fc2(out)
      #out = self.softmax(out)
      return out




def findFiles():
  file1=files.upload()
  file2=files.upload()
  return file1,file2
files=findFiles()
print(files)

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of words per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]
AllLines=""
for file in files:
  print (file)
  for filename, content in file.items():
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
    AllLines=AllLines+" "+' '.join(lines)

n_categories = len(all_categories)

def findAllBigrams(content):
  bigrams=list(ngrams(content, 2, pad_right=True, right_pad_symbol=' '))
  noDuplicate = list(dict.fromkeys(bigrams))
  return (noDuplicate)

all_bigrams=findAllBigrams(AllLines.lower())

FDM = FeedforwardNeuralNetModel(len(all_bigrams), 10, 2)

def bigramToIndex(bigram):
  if bigram in all_bigrams:
    return all_bigrams.index(bigram)
  else:
    return (-1)

print(bigramToIndex(("l","o")))
# Just for demonstration, turn a bigram into a <1 x n_bigrams> Tensor
def bigramToTensor(bigram):
    tensor = torch.zeros(1, all_bigrams.len())
    n=bigramToIndex(bigram)
    if(n>-1):
      tensor[0][n] = 1
    return tensor

# Turn a line into a <line_length_ofBigrams x 1 x n_bigrams>,
# or an array of one-hot vectors
def lineToTensor(line):
  Linebigrams=list(ngrams(line, 2, pad_right=True, right_pad_symbol=' '))
  tensor = torch.zeros( len(all_bigrams))
  for i in range(len(Linebigrams)):
    n=bigramToIndex(Linebigrams[i])
    if(n>-1):
      tensor[n] += 1
  return tensor

#print(lineToTensor("Hello wowowowo".lower()))


def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])

    category_tensor=torch.tensor([all_categories.index(category)]).long()

    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(2):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)



tensor([1])
tensor([-2.5000, -4.2120])
tensor([[-2.5000, -4.2120]])
tensor(1.8779)


Saving Spanish.txt to Spanish.txt


Saving English.txt to English.txt
({'Spanish.txt': b"Abana\nAbano\nAbarca\nAbaroa\nAbascal\nAbasolo\nAbel\nAbell\xc3\xb3\nAberquero\nAbreu\nAcosta\nAgramunt\nAiza\nAlamilla\nAlbert\nAlbuquerque\nAldana\nAlfaro\nAlvarado\n\xc3\x81lvarez\nAlves\nAmador\nAndreu\nAnt\xc3\xbanez\nAqua\nAquino\nAra\xc3\xbajo\nAraullo\nAraya\nArce\nArechavaleta\nArena\nAritza\nArmando\nArreola\nArriola\nAsis\nAsturias\nAvana\nAzarola\nBanderas\nBarros\nBasurto\nBautista\nBello\nBelmonte\nBengochea\nBenitez\nBerm\xc3\xbadez\nBlanco\nBlanxart\nBol\xc3\xadvar\nBonaventura\nBosque\nBustillo\nBusto\nBustos\nCabello\nCabrera\nCampo\nCampos\nCapello\nCardona\nCaro\nCasales\nCastell\nCastellano\nCastillion\nCastillo\nCastro\nChavarr\xc3\xada\nChavez\nCol\xc3\xb3n\nCosta\nCrespo\nCruz\nCu\xc3\xa9llar\nCuevas\nD'cruz\nD'cruze\nDe la cruz\nDe la fuente\nDel bosque\nDe leon\nDelgado\nDel olmo\nDe santigo\nD\xc3\xadaz\nDominguez\nDuarte\nDurante\nEchevarr\xc3\xada\nEcheverr\xc3\xada\nElizondo\nEscamilla\nEsc\xc3\xa1rcega\

In [ ]:


# Define hyperparameters
n_epochs = 100
lr=0.01

criterion = nn.L1Loss()

optimizer = torch.optim.SGD(FDM.parameters(), lr=0.1)

for epoch in range(1, n_epochs + 1):
  category, line, category_tensor, line_tensor = randomTrainingExample()
  optimizer.zero_grad() # Clears existing gradients from previous epoc
  input_seq=line_tensor
  print(category_tensor)
  output = FDM(input_seq)
  loss = criterion(output.view(1,2), category_tensor)
  loss.backward()
  optimizer.step() # Updates the weights accordingly
  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

def predict(input_line, n_predictions=2):
    print('\n> %s' % input_line)
    line_tensor = lineToTensor(input_line)
    with torch.no_grad():
        output = FDM(line_tensor)
        print("Out put is:")
        print(output)
        predictions = []
        result=torch.argmax(output)
        print(result)
        print(result.item())
        category_index = result.item()
        print(all_categories[category_index])


tensor([1])
tensor([1])
tensor([1])
tensor([0])
tensor([0])
tensor([1])
tensor([0])
tensor([0])
tensor([0])
tensor([1])
Epoch: 10/100............. Loss: 0.8160
tensor([1])
tensor([1])
tensor([1])
tensor([0])
tensor([0])
tensor([1])
tensor([0])
tensor([1])
tensor([1])
tensor([1])
Epoch: 20/100............. Loss: 0.0838
tensor([0])
tensor([1])
tensor([1])
tensor([0])
tensor([0])
tensor([0])
tensor([1])
tensor([1])
tensor([1])
tensor([0])
Epoch: 30/100............. Loss: 0.8918
tensor([0])
tensor([1])
tensor([1])
tensor([0])
tensor([0])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
Epoch: 40/100............. Loss: 0.0659
tensor([0])
tensor([1])
tensor([0])
tensor([0])
tensor([0])
tensor([1])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
Epoch: 50/100............. Loss: 0.1789
tensor([1])
tensor([1])
tensor([0])
tensor([0])
tensor([1])
tensor([0])
tensor([1])
tensor([1])
tensor([0])
tensor([1])
Epoch: 60/100............. Loss: 0.7586
tensor([0])
tensor([0])
tensor([0])
tens

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [ ]:
!git clone https://github.com/coqui-ai/TTS/

Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Counting objects: 100% (2774/2774), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 32844 (delta 2725), reused 2701 (delta 2701), pack-reused 30070 (from 1)
Receiving objects: 100% (32844/32844), 166.14 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (23844/23844), done.


In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
import evaluate
import torch.nn.functional as F

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large").to(device)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# Load the XSum dataset (subset for demonstration)
dataset = load_dataset("xsum", split='train[:100]')
rouge = evaluate.load("rouge")

# Compute reward using ROUGE scores
def compute_reward(preds, references):
    rouge_scores = rouge.compute(predictions=preds, references=references)
    reward = rouge_scores["rouge1"]  # Use ROUGE-1 F1 score as reward
    return reward

# Supervised fine-tuning function (optional warm-up)
def supervised_fine_tune(model, tokenizer, dataset, epochs=3, lr=5e-5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for i, data in enumerate(dataset):
            input_text = data['document']
            reference_summary = data['summary']

            # Prepare inputs and labels
            inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(device)
            labels = tokenizer(reference_summary, return_tensors="pt", max_length=128, truncation=True, padding=True).input_ids.to(device)

            # Forward pass
            outputs = model(input_ids=inputs['input_ids'], labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 10 == 0:
                print(f"Step {i}, Loss: {loss.item()}")

        print(f"Epoch {epoch + 1} completed. Total Loss: {total_loss}")

# Fine-tuning with REINFORCE and Baseline Reward Smoothing
def fine_tune_with_reinforce(model, tokenizer, dataset, epochs=3, lr=5e-5, baseline_reward=0.1):

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        print(f"Starting Epoch {epoch + 1}/{epochs}")

        for i, data in enumerate(dataset):
            input_text = data['document']
            reference_summary = data['summary']

            # Tokenize inputs
            inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(device)

            # Generate summary
            generated_ids = model.generate(
                inputs['input_ids'], max_length=60, min_length=10, length_penalty=2.0, num_beams=4
            )
            generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            # Compute reward
            reward = compute_reward([generated_summary], [reference_summary])
            smoothed_reward = max(reward - baseline_reward, 1e-3)  # Reward smoothing and baseline subtraction

            # Compute log probabilities of the generated summary
            outputs = model(input_ids=inputs['input_ids'], labels=generated_ids)
            logits = outputs.logits
            log_probs = F.log_softmax(logits, dim=-1)

            # Policy gradient loss
            loss = -smoothed_reward * log_probs.mean()
            total_loss += loss.item()

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 10 == 0:
                print(f"Step {i}, Loss: {loss.item()}, Reward: {reward}, Smoothed Reward: {smoothed_reward}")

        print(f"Epoch {epoch + 1} completed. Total Loss: {total_loss}")

# Evaluate the model
def evaluate_model(model, tokenizer, dataset):
    """
    Evaluate the fine-tuned model using ROUGE scores.
    """
    model.eval()
    predictions = []
    references = []

    for data in dataset:
        input_text = data['document']
        reference_summary = data['summary']

        # Tokenize input
        inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(device)

        # Generate summary
        generated_ids = model.generate(
            inputs['input_ids'], max_length=60, min_length=10, length_penalty=2.0, num_beams=4
        )
        generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        predictions.append(generated_summary)
        references.append(reference_summary)

    # Compute ROUGE scores
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    print("Evaluation ROUGE Scores:")
    for key, value in rouge_scores.items():
        print(f"{key}: {value}")

# Run the training process
train_data = dataset.select(range(50))  # Subset for fine-tuning
val_data = dataset.select(range(50, 100))  # Subset for evaluation

# Optional supervised fine-tuning
supervised_fine_tune(model, tokenizer, train_data, epochs=1, lr=5e-5)

# Reinforce fine-tuning with baseline and smoothing
fine_tune_with_reinforce(model, tokenizer, train_data, epochs=3, lr=5e-5, baseline_reward=0.1)

# Evaluate the fine-tuned model
evaluate_model(model, tokenizer, val_data)


In [ ]:
print("Model: facebook/bart-large-cnn\n"
      "ROUGE-1: 0.1726\n"
      "ROUGE-2: 0.0302\n"
      "ROUGE-L: 0.1537\n"
      "BLEU: 0.0000\n"
      "Average Semantic Similarity: 0.5213\n\n"
      "Model: t5-small\n"
      "ROUGE-1: 0.1856\n"
      "ROUGE-2: 0.0221\n"
      "ROUGE-L: 0.1315\n"
      "BLEU: 0.0000\n"
      "Average Semantic Similarity: 0.4582")
print("Model: facebook/bart-large-cnn\n"
      "ROUGE-1: 0.1726\n"
      "ROUGE-2: 0.0302\n"
      "ROUGE-L: 0.1537\n"
      "BLEU: 0.0000\n"
      "Average Semantic Similarity: 0.5213\n\n"
      "Model: t5-small\n"
      "ROUGE-1: 0.1856\n"
      "ROUGE-2: 0.0221\n"
      "ROUGE-L: 0.1315\n"
      "BLEU: 0.0000\n"
      "Average Semantic Similarity: 0.4582")


Model: facebook/bart-large-cnn
ROUGE-1: 0.1726
ROUGE-2: 0.0302
ROUGE-L: 0.1537
BLEU: 0.0000
Average Semantic Similarity: 0.5213

Model: t5-small
ROUGE-1: 0.1856
ROUGE-2: 0.0221
ROUGE-L: 0.1315
BLEU: 0.0000
Average Semantic Similarity: 0.4582
Model: facebook/bart-large-cnn
ROUGE-1: 0.1726
ROUGE-2: 0.0302
ROUGE-L: 0.1537
BLEU: 0.0000
Average Semantic Similarity: 0.5213

Model: t5-small
ROUGE-1: 0.1856
ROUGE-2: 0.0221
ROUGE-L: 0.1315
BLEU: 0.0000
Average Semantic Similarity: 0.4582


In [ ]:

predict('The ')
predict('el nomre es')
predict('Hoy el clima está agradable')
predict('The weather is nice today.')
predict('I love learning new languages.')
predict('Me encanta aprender nuevos idiomas.')
predict('This is a great opportunity.')
predict('Esta es una gran oportunidad.')
predict('Artificial intelligence is fascinating.')
predict('La inteligencia artificial es fascinante.')
predict('Can I get a coffee, please?')
predict('¿Puedo pedir un café, por favor?')


> The 
Out put is:
tensor([0.4901, 0.4941])
tensor(1)
1
English

> el nomre es
Out put is:
tensor([0.4573, 0.4589])
tensor(1)
1
English

> Hoy el clima está agradable
Out put is:
tensor([0.3839, 0.4054])
tensor(1)
1
English

> The weather is nice today.
Out put is:
tensor([0.5810, 0.5698])
tensor(0)
0
Spanish

> I love learning new languages.
Out put is:
tensor([0.5062, 0.4796])
tensor(0)
0
Spanish

> Me encanta aprender nuevos idiomas.
Out put is:
tensor([0.3729, 0.3899])
tensor(1)
1
English

> This is a great opportunity.
Out put is:
tensor([0.4408, 0.4320])
tensor(0)
0
Spanish

> Esta es una gran oportunidad.
Out put is:
tensor([0.3696, 0.3806])
tensor(1)
1
English

> Artificial intelligence is fascinating.
Out put is:
tensor([0.3550, 0.4039])
tensor(1)
1
English

> La inteligencia artificial es fascinante.
Out put is:
tensor([0.1976, 0.2602])
tensor(1)
1
English

> Can I get a coffee, please?
Out put is:
tensor([0.4563, 0.4608])
tensor(1)
1
English

> ¿Puedo pedir un café, por fav